# Actions - Sending Actions to the Create3 

Actions are one of the communication types in ROS 2 and are intended for long running tasks. They consist of three parts: a goal, feedback, and a result. Action clients send goal requests to action servers. Action servers send goal feedback and results to action clients.

To do this from the command-line, we use the command `ros2 action send_goal <action_name> <action_type> <values>`. 

The `<action_name>` argument is just like the `<topic_name>` used in the previous cases.

The `<action_type>` argument is similar to the `<msg_type>` argument and defines the category of action you are communicating to.

The `<values>` argument is similar to the `'<args>'` argument is the actual data you’ll pass to the topic. Let's set it to `"{}"`.

In this case let's tell our Create3 to Undock itself, this will only work if the Create3 is currently docked (i.e. connected to its charger.)

```
<action_name> = /undock
<action_type> = irobot_create_msgs/action/Undock
<values> = "{}"
```

Note: Typically the `ros2 action send_goal` command is called from the command-line of a terminal, here we use the `%%bash` magic word to simulate the command line in this jupyter notebook.

In [ ]:
%%bash

ros2 action send_goal /undock irobot_create_msgs/action/Undock "{}"

Sending this command will undock your Create3, if it is currently on its dock. If the Create3 is currently undocked, this command will do nothing.

An example of how to send an action to the Create3 in python is provided below.

## Undock Action Client Example

In [ ]:
import rclpy
from rclpy.action import ActionClient
from rclpy.node import Node

from irobot_create_msgs.action import Undock


class UndockingActionClient(Node):
    '''
    Purpose
    -------
    A simple action client to communicate to the /undock action, this will
    only work if the Create3 is currently docked.
    '''

    def __init__(self):
        super().__init__('undocking_action_client')
        self._action_client = ActionClient(self, Undock, 'undock')

    def send_goal(self):
        goal_msg = Undock.Goal()
        print(goal_msg)

        self._action_client.wait_for_server()

        self._send_goal_future = self._action_client.send_goal_async(goal_msg)

        self._send_goal_future.add_done_callback(self.goal_response_callback)

    def goal_response_callback(self, future):
        goal_handle = future.result()
        if not goal_handle.accepted:
            self.get_logger().info('Goal rejected :( Check that the Create3 is currently docked')
            return

        self.get_logger().info('Goal accepted :)')

        self._get_result_future = goal_handle.get_result_async()
        self._get_result_future.add_done_callback(self.get_result_callback)

    def get_result_callback(self, future):
        result = future.result().result
        self.get_logger().info('Result: {0}'.format(result))
        rclpy.shutdown()


def main(args=None):
    rclpy.init(args=args)

    action_client = UndockingActionClient()

    action_client.send_goal()

    rclpy.spin(action_client)


if __name__ == '__main__':
    main()
